In [10]:
import pickle
import pandas as pd
import numpy as np
import sys
from IPython.display import display, clear_output
from datetime import datetime
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import os
from scipy.sparse import csr_matrix
from scipy.optimize import minimize

Филигранно распикливаем ([с возможной утечкой памяти:)](https://stackoverflow.com/questions/7395542/is-explicitly-closing-files-important))

In [11]:
players_dict = pickle.load(open('data/chgk/players.pkl', 'rb'))
results_dict = pickle.load(open('data/chgk/results.pkl', 'rb'))
tournaments_dict = pickle.load(open('data/chgk/tournaments.pkl', 'rb'))

В тренировочном наборе оставляем только турниры, у которых `dateStart` 2019, в тестовый набор - турниры, у которых `dateStart` 2020

Фильтруем `results`, оставляем только те, где есть `mask` в ключах (без повопросных результатов обучать силу проблематично)

In [12]:
results_filtered_train = {}
results_filtered_test = {}
for k, res in results_dict.items():
    t_res_new = []
    for t_res in results_dict[k]:
        if 'mask' in t_res.keys():
            if t_res['mask'] is not None:
                if 'X' not in t_res['mask'] and '?' not in t_res['mask']:
                    t_res_new.append(t_res)
    if t_res_new:
        if tournaments_dict[k]['dateStart'].startswith('2019'):
            results_filtered_train[k] = t_res_new
        if tournaments_dict[k]['dateStart'].startswith('2020'):
            results_filtered_test[k] = t_res_new

Также понадобится правильное число вопросов в каждом турнире, чтобы фильтровать результаты по длине маски
(иначе получится так, что разные команды в рамках одного турнира ответили на разное количество вопросов, а значит не получится восстановить однозначное соответствие `команда`-`вопрос`) 

In [13]:
tournament_question_count = {}

for k, v in results_filtered_train.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])
    
for k, v in results_filtered_test.items():
    tournament_question_count[k]= max([len(t_res['mask']) for t_res in v])

In [14]:
results_filtered_train_len = {}
results_filtered_test_len = {}
train_players = set()
for k, v in results_filtered_train.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            train_players.update([m['player']['id'] for m in t_res['teamMembers']])
            t_res_.append(t_res)
    results_filtered_train_len[k] = t_res_

for k, v in results_filtered_test.items():
    t_res_ = []
    for t_res in v:
        if len(t_res['mask']) == tournament_question_count[k]:
            t_res_.append(t_res)
    results_filtered_test_len[k] = t_res_

Делая наивное предположение что если команда ответила на вопрос, то каждый игрок в этой команде ответил на вопрос, собираем большую таблицу взаимодействий `игрок-вопрос-[ответил/не ответил]`

In [15]:
pq_df = pd.DataFrame(columns=['pid', 'qid', 'tournament', 'tid', 'res'])

pid = []
qid = []
res = []
tournament = []
tid = []
for k, v in results_filtered_train_len.items():
    for t_res in v:
        members = [m['player']['id'] for m in t_res['teamMembers']]
        t_pid = list(itertools.chain.from_iterable(itertools.repeat(m, tournament_question_count[k]) for m in members))
        pid.extend(t_pid)
        t_qid = [f'{k}_{i}' for i in range(tournament_question_count[k])] * len(members)
        qid.extend(t_qid)
        tid.extend([t_res['team']['id']] * len(t_qid))
        tournament.extend([k] * len(t_qid))
        res.extend(list(map(int, t_res['mask'])) * len(members))

pq_df['pid'] = np.int32(pid)
pq_df['qid'] = qid
pq_df['tournament'] = tournament
pq_df['tid'] = tid
pq_df['res'] = np.int8(res)
pq_df['qid'] = pq_df['qid'].astype(str)
pq_df['pid'] = pq_df['pid'].astype(str)

Словарь `tournament_results_train` и `tournament_results_t_test` нужен для тестирования полученных моделей

In [16]:
tournament_results_train = {}
for tourn_id, t_res in results_filtered_train_len.items():
    tids, names, positions, qtotals, members = [], [], [], [], []
    for team in t_res:
        members.append([m['player']['id'] for m in team['teamMembers']])
        tids.append(team['team']['id'])
        names.append(team['team']['name'])
        positions.append(team['position'])
        qtotals.append(team['questionsTotal'])
        
    tournament_results_train[tourn_id] = pd.DataFrame(
        {
        'tid': tids,
        'name': names,
        'position': positions,
        'qtotal': qtotals,
        'members': members
        }
    ).sort_values(by='position')
    

tournament_results_test = {}
for tourn_id, t_res in results_filtered_test_len.items():
    tids, names, positions, qtotals, members = [], [], [], [], []
    for team in t_res:
        t_members = []
        for m in team['teamMembers']:
            if m['player']['id'] in train_players:
                t_members.append(m['player']['id'])
        if t_members:
            members.append(t_members)
            tids.append(team['team']['id'])
            names.append(team['team']['name'])
            positions.append(team['position'])
            qtotals.append(team['questionsTotal'])
    tournament_results_test[tourn_id] = pd.DataFrame(
        {
        'tid': tids,
        'name': names,
        'position': positions,
        'qtotal': qtotals,
        'members': members
        }
    ).sort_values(by='position')

### Логистическая регрессия

#### Предположения:

- игрок ответил на вопрос == команда в которой был игрок ответила на вопрос

- вероятность того, что игрок ответит на вопрос зависит от сложности вопроса и умений игрока, и никак не зависит от того в какой он команде

__Тогда__:

Событие $A_{pq}$ == p-игрок ответил на q-ый вопрос

В таком случае

$P(A_{pq}) = \sigma(skill_p + difficult_q)$

Таким образом, из повопросного датасета можно выучить `skill_p` и `difficult_q` (просто записав лолгосс и минимизировать его градиентным спуском)

Однако, очень удобно (для того чтобы использовать библиотечные солверы и прочие радости) кодировать $A_{pq}$ с помощью `OneHotEncoding`. В таком случае, 
если $x_i$ = `[0, 0, ..., [1 на месте p], 0, 0, ... [1 на месте q]]`, умножить его скалярно на вектор `np.concat([skills, difficults])` это то же самое, что сложить `skill_p + difficult_q`

In [17]:
encoder = OneHotEncoder(sparse=True, categories=[pq_df['pid'].unique().tolist(), pq_df['qid'].unique().tolist()])
encoder.fit(pq_df[['pid', 'qid']])
pq_df_oh = encoder.transform(pq_df[['pid', 'qid']])
clf = LogisticRegression(solver='lbfgs', n_jobs=10)
clf.fit(pq_df_oh, pq_df['res'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=10, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Строим рейтинг

In [18]:
def get_tournaments_rating_n_corrs(encoder, player_q_df, weights, t_results_test):
    t_results_test_ = t_results_test.copy()
    feature_names = encoder.get_feature_names(pq_df.columns[:2])
    players = feature_names[[fname.startswith('pid') for fname in feature_names]]
    players = np.array(list(map(lambda x: np.int32(x.replace('pid_','')), players)))
    players_scores = pd.DataFrame({'player_id': players, 'score': weights[:len(players)]}).set_index('player_id')
    corrs_spearman = []
    corrs_kendall = []
    for t_id, t_result in t_results_test_.items():
        t_result['predicted_score'] = t_result['members'].apply(lambda x: players_scores.loc[x]['score'].mean())
        corrs_spearman.append(np.abs(t_result[['position', 'predicted_score']].corr(method='spearman')['position']['predicted_score']))
        corrs_kendall.append(np.abs(t_result[['position', 'predicted_score']].corr(method='kendall')['position']['predicted_score']))
    return t_results_test_, np.mean(corrs_spearman), np.mean(corrs_kendall)

In [19]:
t_results_pred, spearman, kendall = get_tournaments_rating_n_corrs(encoder, pq_df, clf.coef_[0], tournament_results_test)

In [20]:
print(spearman)
print(kendall)

0.7444559048119628
0.5867546216490609


### EM - схема

Поскольку ЧГК - командная игра, кажется не слишком разумным что если игрок взял вопрос вместе с какой-то командой, то он сам ответил на этот вопрос. Нужно как-то учитывать выступление одного и того же игрока в разных командах, другими словами, как и сказано в задании, переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды

__Предлагаемая EM-схема__:

Будем использовать результат игрока $p$ на вопросе $q$ в качестве скрытой переменной (`res_pq`). Тогда

* __E-шаг__:

На $E$-шаге оцениваем мат-ожидание выбранной скрытой переменной. Предположим, что если хотя бы один игрок ответил на вопрос верно, то команда ответит на вопрос верно.

Тогда

$$\mathop{\mathbb{E}}(res_{pq}) = {p(A_{pq} | A_{tq})} = \frac{\sigma(skill_p + difficult_q)}{1 - \prod_{p \in t}(1 - \sigma(skill_p + difficult_q))}$$, 
если команда ответила на вопрос, и $0$, если не ответила


* __M-шаг__:

На $M$-шаге просто учим логистическую регрессию на оцененные матожидания (правда я так и не понял как модель из `sklearn` заставить минимизировать лосс с мягкими метками, поэтому в качестве "обучения логистической регрессии" просто будем делать несколько шагов в сторону градиента правдоподобия

In [21]:
def sigmoid(X, weights):
    return 1 / (1 + np.exp(-X.dot(weights)))

def log_likelihood(expectations, pq_df_oh, weights):
    sigma = sigmoid(pq_df_oh, weights)
    return np.sum(expectations * np.log(sigma) + (1 - expectations) * np.log(1 - sigma))
    
def weights_gradient(weights, expectations, pq_df_oh):
    return (csr_matrix(expectations - sigmoid(pq_df_oh, weights)).dot(pq_df_oh)).toarray()[0] / np.array((pq_df_oh != 0).sum(axis=0)).ravel()

def Expectation(weights, pq_df, pq_df_oh):
    pq_df_ = pq_df.copy()
    sigma = sigmoid(pq_df_oh, weights)
    pq_df_['sigma'] = sigma
    pq_df_['one_sigma'] = 1 - sigma
    prods_series = 1 - pq_df_.groupby(['tournament', 'tid', 'qid'])['one_sigma'].prod().rename('prod')
    pq_df_ = pq_df_.merge(prods_series, on=['tournament', 'tid', 'qid'])
    expectations = (pq_df_['sigma'] / (pq_df_['prod'])).values
    expectations[pq_df_['res'] == 0] = 0    
    return expectations

def Maximization(expectations, pq_df_oh, weights):
    weights_ = weights
    for i in range(30):
        weights_ += 1.5 * weights_gradient(weights_, expectations, pq_df_oh)
    return weights_

In [22]:
weights = np.random.rand(pq_df_oh.shape[1])
em_df = pd.DataFrame(columns=['iter', 'log_ll', 'Kendall_corr', 'Spearman_corr'])
for i in range(10):
    expectations = Expectation(weights, pq_df, pq_df_oh)
    weights = Maximization(expectations, pq_df_oh, weights)
    rating, corr_spearman, corr_kendall = get_tournaments_rating_n_corrs(encoder, pq_df, weights, tournament_results_test)
    em_df = em_df.append(
       pd.DataFrame(
        {
        'iter': [i + 1],
        'log_ll': [log_likelihood(expectations, pq_df_oh, weights) / pq_df_oh.shape[0]],
        'Kendall_corr': [corr_kendall],
        'Spearman_corr' : [corr_spearman]
    }), ignore_index=True)
    clear_output()
    display(em_df)

,iter,log_ll,Kendall_corr,Spearman_corr
0,1,-0.570342,0.580205,0.735686
1,2,-0.421324,0.589948,0.746765
2,3,-0.355249,0.591595,0.747636
3,4,-0.331625,0.587803,0.743784
4,5,-0.323231,0.586202,0.741593
5,6,-0.320162,0.584348,0.739779
6,7,-0.318998,0.583620,0.738872
7,8,-0.318538,0.583471,0.738921
8,9,-0.318352,0.583371,0.738919
9,10,-0.318276,0.583364,0.738748


### Рейтинг лист турниров по сложности вопросов

EZ

In [25]:
def get_tournaments_difficult_rating(weights, pq_df, encoder):
    feature_names = encoder.get_feature_names(pq_df.columns[:2])
    questions = feature_names[[fname.startswith('qid') for fname in feature_names]]
    questions = np.array(list(map(lambda x: x.replace('qid_',''), questions)))
    questions_scores = pd.DataFrame({'qid': questions , 'score': weights[-len(questions):]})
    train_tournaments_diffs = pq_df.merge(questions_scores, on='qid').groupby('tournament')['score'].mean()
    diffs = []
    names = []
    tids = []
    for tournament_id in train_tournaments_diffs.keys():
        diffs.append(train_tournaments_diffs[tournament_id])
        names.append(tournaments_dict[tournament_id]['name'])
        tids.append(tournament_id)
    tournaments_diff_rating = pd.DataFrame({
        'tid' : tids,
        'name': names,
        'diff': diffs
    }).sort_values(by='diff')
    tournaments_diff_rating['place'] = tournaments_diff_rating.reset_index().index + 1
    tournaments_diff_rating = tournaments_diff_rating[['name', 'place']].set_index('place')
    return tournaments_diff_rating

In [28]:
rating_tournaments = get_tournaments_difficult_rating(weights, pq_df, encoder)
rating_tournaments.head(25)

,name
place,
1,Чемпионат Санкт-Петербурга. Первая лига
2,Угрюмый Ёрш
3,Первенство правого полушария
4,Чемпионат Мира. Этап 2 Группа С
5,Воображаемый музей
6,Чемпионат Мира. Финал. Группа С
7,Записки охотника
8,Чемпионат Мира. Этап 3. Группа С
9,Чемпионат Санкт-Петербурга. Высшая лига


In [29]:
rating_tournaments.tail(25)

,name
place,
592,Открытый кубок УМЭД
593,Большая перемена
594,Школьный Синхрон-lite. Выпуск 3.4
595,Парный асинхронный турнир ChGK is...
596,Кубок княгини Ольги среди школьников
597,(а)Синхрон-lite. Лига старта. Эпизод VI
598,Лига вузов. IV тур
599,Чемпионат МГУ. Открытая лига. Первый игровой день
600,Школьный Синхрон-lite. Выпуск 2.3


Как видно, всякие школьные и несерьезные турниры попадают в конец рейтинга, а Чемпионаты мира и турниры Первой лиги в начало